# Using Blueprints with Data Projects

## Introduction

Data projects in Amazon Bedrock data automation (BDA) provide a easy way of grouping your standard and customt output configuration for processing files. You can create a BDA project and use the ARN of the project to call the `InvokeDataAutomationAsync` API. BDA processes the input file automatically using the configuration settings defined in that project. Output is then generated based on the project's configuration. You can use a single project resource for multiple file types. 

You can also configure a project with Blueprints for documents (or images), to define custom output. In this notebook, we will explore the capability of using project with blueprints for processing documents. We will start with creating a project with a single blueprint and progress to add multiple blueprints (preexisting and custom) to process file with multiple documents.

## Using projects with custom output

You can configure custom output for documents by adding a new blueprint (or a preexisting blueprint from BDA global catalog) to the BDA project. If your use case has different kinds of documents then you can use  multiple blueprints for the different document kinds with the project.

**Note: A project chan have up to 40 document blueprints attached.**

When you attach multiple blueprints with a project, BDA would automatically find an appropriate blueprint matching using the input document. Once a matching blueprint is found, BDA generates custom output using that blueprint.

Let's go through the steps to creating a project and attaching a set of blueprints to process different file types.

## Prerequisites

In [ ]:
pip install "boto3>=1.35.76" PyPDF2 --upgrade -qq

## Setup

Before we get to the part where we invoke BDA with our sample artifacts, let's setup some parameters and configuration that will be used throughout this notebook

In [ ]:
import boto3
import json
import pprint
from IPython.display import JSON, display, HTML, Markdown, IFrame
import IPython.display as display
from sagemaker import get_execution_role


%load_ext autoreload
%autoreload 1
import sys
sys.path.append('..')
%aimport utils.helper_functions

from utils.helper_functions import read_s3_object, create_sample_file, get_bucket_and_key,wait_for_completion, display_html

default_execution_role = get_execution_role()
region_name = 'us-west-2'
# Initialize Bedrock Data Automation client
bda_client = boto3.client('bedrock-data-automation')
bda_runtime_client = boto3.client('bedrock-data-automation-runtime')
s3_client = boto3.client('s3')

bda_s3_input_location = 's3://genai.octankmarkets.com/input'
bda_s3_output_location = 's3://genai.octankmarkets.com/output'
print(f'Using default execution role {default_execution_role}')

## Configure IAM Permissions


The features being explored in the notebook require the following IAM Policies for the role being used. If you're running this notebook within SageMaker Studio in your own Account, update the default execution role for the SageMaker user profile to include the following IAM policies. 

```json
[
    {
        "Sid": "BDACreatePermissions",
        "Effect": "Allow",
        "Action": [
            "bedrock:CreateDataAutomationProject",
            "bedrock:CreateBlueprint"
        ],
        "Resource": "*"
    },
    {
        "Sid": "BDAOProjectsPermissions",
        "Effect": "Allow",
        "Action": [
            "bedrock:CreateDataAutomationProject",
            "bedrock:UpdateDataAutomationProject",
            "bedrock:GetDataAutomationProject",
            "bedrock:GetDataAutomationStatus",
            "bedrock:ListDataAutomationProjects",
            "bedrock:InvokeDataAutomationAsync"
        ],
        "Resource": "arn:aws:bedrock:::data-automation-project/*"
    },
    {
        "Sid": "BDABlueprintPermissions",
        "Effect": "Allow",
        "Action": [
            "bedrock:GetBlueprint",
            "bedrock:ListBlueprints",
            "bedrock:UpdateBlueprint",
            "bedrock:DeleteBlueprint"
        ],
        "Resource": "arn:aws:bedrock:::blueprint/*"
    }
]

### View Sample Document

In [ ]:
IFrame("examples/BankStatement.pdf", width=900, height=800)

### Upload sample document to S3
For this lab, we use a CMS 1500 Medical claim for with dummy data to explore the blueprint feature of BDA. 

Note - We will configure BDA to use the sample input from this S3 location, so we need to ensure that BDA has `s3:GetObject` access to this S3 location. If you are running the notebook in your own AWS Account, ensure that the SageMaker Execution role configured for this JupyterLab app has the right IAM permissions.

In [ ]:
%load_ext autoreload
%autoreload 1
import sys
sys.path.append('..')
%aimport utils.helper_functions 
from utils.helper_functions import wait_for_job_to_complete, read_s3_object, create_sample_file, get_bucket_and_key, wait_for_completion, display_html

input_bucket, input_prefix = get_bucket_and_key(bda_s3_input_location)
local_file_name = 'examples/BankStatement.pdf'
s3_file_name = 'BankStatement.pdf'
s3_response = s3_client.upload_file(local_file_name, input_bucket,
                                    f'{input_prefix}/{s3_file_name}')

### Invoke Blueprint Recommendation
With our sample ready, we can have BDA recommend a blueprint for our sample document from the sample set.

In [ ]:
from utils.helper_functions import invoke_blueprint_recommendation_async, get_blueprint_recommendation
import json
inputConfiguration = {
    "inputDataConfiguration":{
        "s3Uri":f'{bda_s3_input_location}/{s3_file_name}'
    }
}
response = invoke_blueprint_recommendation_async(bda_client=bda_client,
                                      region_name=region_name, 
                                      credentials = boto3.Session().get_credentials().get_frozen_credentials(),
                                      payload=json.dumps(inputConfiguration))

job_id = response['jobId']

### Wait for blueprint recommendation results

In [ ]:
status_response = wait_for_completion(
            client=None,
            get_status_function=get_blueprint_recommendation,
            status_kwargs={
                'bda_client': bda_client,
                'job_id': job_id,
                'region_name': region_name,
                'credentials': boto3.Session().get_credentials().get_frozen_credentials(),
            },
            completion_states=['Completed'],
            error_states=['ClientError', 'ServiceError'],
            status_path_in_response='status',
            max_iterations=15,
            delay=30
)

### Identify Blueprint
BDA outputs a blueprint recommendation along with a prompt recommendation that is useful to create a custom blueprint, if needed.

For this example, we will fetch the blueprint that was recommended by BDA.

In [ ]:
blueprint_recommendation = next((result for result in status_response['results'] if result['type'] == 'BLUEPRINT_RECOMMENDATION'),None)

In [ ]:
blueprint_arn = blueprint_recommendation['blueprintRecommendation']['matchedBlueprint']['blueprintArn']

The `create_data_automation_project` API takes a project name, description, stage (LIVE / DEVELOPMENT), the standard output configuration and a custom output configuration as input. We are only focussing on the custom output in this notebook, so we leave the standard output configuration as empty so BDA will use the defaults. Additionally, we use a custom configuration with the arn for the recommended blueprint.

In [ ]:
bda_project_name = 'document-custom-output'
bda_project_stage = 'LIVE'
standard_output_configuration = {
    'document': {
        'extraction': {
            'granularity': {
                'types': [
                    'DOCUMENT', 'PAGE'
                ]
            },
            'boundingBox': {
                'state': 'ENABLED'
            }
        },
        'generativeField': {
            'state': 'ENABLED'
        },
        'outputFormat': {
            'textFormat': {
                'types': [
                    'MARKDOWN'
                ]
            },
            'additionalFileFormat': {
                'state': 'ENABLED'
            }
        }
    }
}

custom_output_configuration={
    'blueprints': [
            {
                'blueprintArn': blueprint_arn
            },
        ]
}

### Create data automation project

Check if the project with the given name already exists, if no create a new project with given configuration otherwise update the data project with the given configuration and stage

In [ ]:
list_project_response = bda_client.list_data_automation_projects(
    projectStageFilter=bda_project_stage)

project = next((project for project in list_project_response['projects']
               if project['projectName'] == bda_project_name), None)

if not project:
    response = bda_client.create_data_automation_project(
        projectName=bda_project_name,
        projectDescription='A BDA Project for Standard Output from Document',
        projectStage=bda_project_stage,
        standardOutputConfiguration=standard_output_configuration,
        customOutputConfiguration=custom_output_configuration)
else:
    response = bda_client.update_data_automation_project(
        projectArn=project['projectArn'],
        standardOutputConfiguration=standard_output_configuration,
        customOutputConfiguration=custom_output_configuration)

project_arn = response['projectArn']

### Wait for create/update data project operation completion

In [ ]:
status_response = wait_for_completion(
            client=bda_client,
            get_status_function=bda_client.get_data_automation_project,
            status_kwargs={'projectArn': project_arn},
            completion_states=['COMPLETED'],
            error_states=['FAILED'],
            status_path_in_response='project.status',
            max_iterations=15,
            delay=30
)

In [ ]:
status_response = wait_for_completion(
            client=None,
            get_status_function=get_blueprint_recommendation,
            status_kwargs={
                'bda_client': bda_client,
                'job_id': job_id,
                'region_name': region_name,
                'credentials': boto3.Session().get_credentials().get_frozen_credentials(),
            },
            completion_states=['Completed'],
            error_states=['ClientError', 'ServiceError'],
            status_path_in_response='status',
            max_iterations=15,
            delay=30
)

### Invoke Data Automation Async
With the data project configured, we can now invoke data automation for our sample document.

In [ ]:
response = bda_runtime_client.invoke_data_automation_async(
    inputConfiguration={
        's3Uri': f'{bda_s3_input_location}/{s3_file_name}'
    },
    outputConfiguration={
        's3Uri': bda_s3_output_location
    },
    dataAutomationConfiguration={
        'dataAutomationArn': project_arn,
        'stage': 'LIVE'
    }
)

invocationArn = response['invocationArn']
print(f'Invoked data automation job with invocation arn {invocationArn}') 

### Get Data Automation Status

We can check the status and monitor the progress of the Invocation job using the `GetDataAutomationStatus`. This API takes the invocation arn we retrieved from the response to the `InvokeDataAutomationAsync` operation above.

The invocation job status moves from `Created` to `InProgress` and finally to `Success` when the job completes successfully, along with the S3 location of the results. If the job encounters and error the final status is either `ServiceError` or `ClientError` with error details

In [ ]:
status_response = wait_for_completion(
            client=bda_client,
            get_status_function=bda_runtime_client.get_data_automation_status,
            status_kwargs={'invocationArn': invocationArn},
            completion_states=['Success'],
            error_states=['ClientError', 'ServiceError'],
            status_path_in_response='status',
            max_iterations=15,
            delay=30
)
if status_response['status'] == 'Success':
    job_metadata_s3_location = status_response['outputConfiguration']['s3Uri']
else:
    raise Exception(f'Invocation Job Error, error_type={status_response["error_type"]},error_message={status_response["error_message"]}')

### Retrieve Job Metadata

In [ ]:
job_metadata = json.loads(read_s3_object(job_metadata_s3_location))
JSON(job_metadata, root='job_metadata', expanded=True)

### Extract Custom output

We can now explore the custom output received from processing document.

In [ ]:
asset_id = 0
custom_output_path = next(item["segment_metadata"][0]["custom_output_path"] 
                                for item in job_metadata["output_metadata"] 
                                if item['asset_id'] == asset_id)
custom_output = json.loads(read_s3_object(custom_output_path))
inference_result = custom_output.get('inference_result')
explainability_info = custom_output.get('explainability_info')[0]

### Extract Standard output

In [ ]:
asset_id=0
standard_output_path = next(item["segment_metadata"][0]["standard_output_path"] 
                                for item in job_metadata["output_metadata"] 
                                if item['asset_id'] == asset_id)
standard_output = json.loads(read_s3_object(standard_output_path))

In [ ]:
JSON(standard_output)

In [ ]:
custom_output

In [ ]:
schema = json.loads(blueprint_recommendation.get('blueprintRecommendation').get('schema'))

scalar_keys = [k for k, v in schema.get('properties').items()
               if v.get('type') != 'array']
table_keys = [k for k, v in schema.get('properties').items()
              if v.get('type') == 'array']

In [ ]:
## copy s3 image to local directory
image_body = 

In [ ]:
!aws s3 cp {standard_output.get('pages')[0]['asset_metadata']['rectified_image']} 'examples/rectified_claim_form.jpeg'

### View Extracted Content

In [ ]:
%load_ext autoreload
%autoreload 1
import sys
sys.path.append('..')
%aimport utils.display_functions 
from utils.display_functions import display_result, create_key_value_box

extraction_result={}
for key in scalar_keys:
    extraction_result[key] = (inference_result.get(key), int(explainability_info.get(key).get('confidence')*100))


display_result(document_image_uri='examples/rectified_claim_form.jpeg',kvpairs=extraction_result)

## Clean up

In [ ]:
!rm examples/rectified_claim_form.jpeg

## Conclusion
In this notebook, we configured a project with Blueprints for documents, to define custom output.

## Next Steps

When processing documents, you might want to use multiple blueprints for different kinds of documents that are passed to your project. A project can have up to 40 document blueprints attached. BDA automatically matches your documents to the appropriate blueprint that's configured in your project, and generates custom output using that blueprint.

As a next step, we will configure a project with multiple blueprints for different kinds of documents. A project can have up to 40 document blueprints attached. We will explore how BDA automatically matches your documents to the appropriate blueprint that's configured in your project, and generates custom output using that blueprint.

In [ ]:
IFrame("examples/sample1_cms-1500-P.pdf", width=900, height=800)